# Megatron 直接使用预训练模型进行预测 (BPE v2)

## 环境准备

准备运行这个笔记本的 Jupyter kernel(**如果已经准备就绪，不要重复执行！**)：


1. 配置一个 Conda 环境作为 Jupyter Kernel

In [1]:
# %conda env update -f environments/environment-ipy.yml

安装完毕后，为该 Notebook 选择这个 Kernel (名为`Megatron_LM-ipy`)

2. 在Kernel所在 Conda 环境中安装 Apex

需要通过 pip 从 github 下载源代码安装：

In [2]:
# %pip install -v -r requirements/apex.txt

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [3]:
%cd ..

/home/Public/Megatron-LM


## 指定 Checkpoints 目录

### 从 S3 下载

- 第一种选择：从 s3 下载

文件比较大，根据实际情况选择下载，**不要重复下载**

如果直接使用 S3 上的模型，需要下载，然后修改超参数, 路径等：

In [ ]:
%%time

import os

HPARAMS_NAME = '117m'  # '345m'
MODEL_NAME = '117m-hmwebmix_191128-bpe_v2' #  '345m-xinliqa-hunyin'
TOKENIZER_TYPE = 'GPT2BPETokenizer_CN'

AWSS3_CKPTS_DIR = os.path.join('s3://huamei/hmgpt2-checkpoints', MODEL_NAME)
LOCAL_CKPTS_DIR = os.path.join('./checkpoints', MODEL_NAME)

# 复制 latest_checkpointed_iteration.txt
!aws s3 cp \
    {AWSS3_CKPTS_DIR} \
    {LOCAL_CKPTS_DIR} \
    --recursive \
    --exclude "*" \
    --include "latest_checkpointed_iteration.txt"

# 下载后读取最新的 checkpoint iter 名称
iteration = open(f'{LOCAL_CKPTS_DIR}/latest_checkpointed_iteration.txt').read()
iteration = int(iteration)
iteration_dir = 'iter_{:07d}'.format(iteration)

awss3_ckpt_dir = os.path.join(AWSS3_CKPTS_DIR, iteration_dir)
local_ckpt_dir = os.path.join(LOCAL_CKPTS_DIR, iteration_dir)

print(f'{awss3_ckpt_dir} ==> {local_ckpt_dir}')
    
# 同步最新的 Checkpiont
!aws s3 sync {awss3_ckpt_dir} {local_ckpt_dir}

#
load_model_dir = LOCAL_CKPTS_DIR
print('load: ', load_model_dir)
print('iteration: ', iteration)

### 直接使用本地

- 另外一个选择：直接使用本地的已有模型

修改超参数, 路径等：

In [4]:
%%time

import os

HPARAMS_NAME = '117m'
TOKENIZER_TYPE = 'SentencePieceTokenizer'

MODEL_NAME = '117m.spm-xinli_qa-hunyin'
LOCAL_CKPTS_DIR = os.path.join('./checkpoints', MODEL_NAME)


# 读取最新的 checkpoint iter 名称
iteration = open(f'{LOCAL_CKPTS_DIR}/latest_checkpointed_iteration.txt').read()
iteration_dir = 'iter_{:07d}'.format(int(iteration))

local_ckpt_dir = os.path.join(LOCAL_CKPTS_DIR, iteration_dir)

load_model_dir = LOCAL_CKPTS_DIR
print('load: ', load_model_dir)
print('iteration: ', iteration)

load:  ./checkpoints/117m.spm-xinli_qa-hunyin
iteration:  60000
CPU times: user 2.47 ms, sys: 452 µs, total: 2.92 ms
Wall time: 1.62 ms


## Environment Variables

- 用哪个/些 GPU?

In [5]:
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_VISIBLE_DEVICES=0


## Importings

In [6]:
import copy
import csv
import json
import math
import os
import random
import sys
import time
from contextlib import closing
from itertools import chain, compress
from functools import partial
from multiprocessing import Pool
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm, trange

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [7]:
args = SimpleNamespace(
    # Model arguments
    # To be updated ...
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=1024,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type=TOKENIZER_TYPE,
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load=load_model_dir,
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=256,
)

In [8]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


In [9]:
HPARAMS_SCHEMA = {
    '117m': dict(
        num_layers=12,
        hidden_size=768,
        num_attention_heads=12,
        max_position_embeddings=1024,
    ),
    '345m': dict(
        num_layers=24,
        hidden_size=1024,
        num_attention_heads=16,
        max_position_embeddings=1024,
    ),
}

# 设置 GPT-2 模型的超参数
for k, v in HPARAMS_SCHEMA[HPARAMS_NAME].items():
    setattr(args, k, v)

In [10]:
display(args)

namespace(DDP_impl='local', attention_dropout=0.1, cache_dir=None, checkpoint_activations=None, checkpoint_num_layers=1, cuda=True, distributed_backend='nccl', dynamic_loss_scale=True, eod_mask_loss=None, finetune=None, fp16=True, greedy=False, hidden_dropout=0.1, hidden_size=768, hysteresis=2, load='./checkpoints/117m.spm-xinli_qa-hunyin', local_rank=None, loss_scale=None, loss_scale_window=1000, make_vocab_size_divisible_by=128, max_position_embeddings=1024, min_scale=1, model_parallel_size=1, no_load_optim=None, no_load_rng=None, num_attention_heads=12, num_layers=12, out_seq_length=256, rank=0, recompute=None, reset_attention_mask=None, reset_position_ids=None, resume_dataloader=None, seed=1234, seq_length=1024, temperature=1.0, tokenizer_model_type='bert-large-uncased', tokenizer_path='./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model', tokenizer_type='SentencePieceTokenizer', top_k=0, top_p=0.0, vocab_size=None, world_size=1)

## Init

### 初始化函数/全局变量

In [11]:
tokenizer = None
model = None

def initialize():
    global model, tokenizer

    # Disable CuDNN.
    torch.backends.cudnn.enabled = False

    # Pytorch distributed.
    initialize_distributed(args)

    # Random seeds for reproducability.
    set_random_seed(args.seed)

    # get the tokenizer
    tokenizer = prepare_tokenizer(args)

    # Model, optimizer, and learning rate.
    model = setup_model(args)

    args.device = torch.cuda.current_device()

    # setting default batch size to 1
    args.batch_size = 1

    assert mpu.get_model_parallel_rank() == 0

### 主进程初始化

In [12]:
%%time

initialize()

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 110516736
global rank 0 is loading checkpoint ./checkpoints/117m.spm-xinli_qa-hunyin/iter_0060000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/117m.spm-xinli_qa-hunyin/iter_0060000/mp_rank_00/model_optim_rng.pt
CPU times: user 5.13 s, sys: 1.67 s, total: 6.8 s
Wall time: 6.82 s


## Inference functions

In [13]:
# restore generating args
def reset_generating_args(args):
    args.recompute=False
    args.top_p=0
    args.top_k=0
    args.temperature=1

def infer_tokens_generative(context_tokens, model, tokenizer):
    with torch.no_grad():
        context_length = len(context_tokens)
        token_stream = get_token_stream(model, [context_tokens], tokenizer, args)   
        for i, (output_tokens, _) in enumerate(token_stream):
            if context_length + i >= args.seq_length:
                break
            ids = output_tokens.cpu().numpy().tolist()[0]
            yield ids[-1]


def infer_text_generative(contex_text, model, tokenizer):
    with torch.no_grad():
        contex_text = contex_text.strip()
        context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
        context_length = len(context_tokens)

        token_stream = get_token_stream(model, [context_tokens], tokenizer, args)

        for i, (output_tokens, _) in enumerate(token_stream):
            if context_length + i >= args.seq_length:
                break
            ids = output_tokens.cpu().numpy().tolist()[0]
            s = tokenizer.DecodeIds([ids[-1]])
            yield s


## 预测试试看

In [14]:
input_texts = [
    # 这几个是 train 数据
#     (
#         '老公说和女老师是闺蜜?觉得老公精神出轨该怎么办?',
#         '与老公关系越来越冷漠。离婚是想过，但离婚结果是伤害三个人，最重的是孩子。我比较敏感，觉得老公精神出轨，或亦是十多年磨合也抹掉对我的在乎，我的一切理所当然归他，他没有危机感。就让为人在就好，其他交流没有都可以。他可以同学，同事（尤其是女同事）一呼百应，对我的情绪和需求却时而不见。我也不知道该不该离?离伤己，伤孩，更伤父母，我不想做做样子的罪人。但我与老公的交流无效，我活在一口枯井里，抑郁得不行，加上单位效益差，想过重读书或考技能证专业，短期又不现实，再八年就退休了。真心累，找不到出口。嫉妒心又强，就连儿子亲近爸爸，我都会嫉恨孩子的做法，（我从不查丈夫，孩子的手机，结果个个都设密码，我有时候找不到手机，想借用都打不开。）压抑这样子愤怒的情绪的非常累。怎么办?',
#     ),
#     (
#         '前任和现在的纠结',
#         '是这样的，我和前任分手了 原因是他真的太好了 我觉得我会耽搁了他 最后逼走了前任 现在我和现任在一起了 可是每隔一段时间我就在痛心和前任的事情 总是忍不住想找前任 有一次忍不住发了个信息给前任 结果间接导致了前任和他的女友分手了（当时我不知道他在拍拖） 现任对我很好 可是老是吵嘴 可是我老是在比较 觉得现任和前任的差别 就不自主的挂念着前任 后悔当初怎么选择逼走他 觉得对不住前任 可是我的第一次阴差阳错的给了现任了 触碰了这个底线 可是我又想和前任一起 现在好纠结 您觉得我是应该好好珍惜现任呢 还是重新争取前任 我知道我现在就是等于三心两意 好贱 可是自己又由不得自己似得 好讨厌这样的自己',
#     ),

    # 这几个是网上随便找的数据
    (
        '男人出轨',
        '我的意思是，出轨者会问自己，我为什么要这么做，被背叛的伴侣会问，你为什么要这样对我?'
    ),
    (
        '我的奇葩婚姻，老公说为了孩子好就是不离婚？',
        '我的婚姻最近好像一塌糊涂，可是老公不离婚，我甚至有想过死？我是不是得了抑郁症？和老公从2017年开始就吵吵闹闹，一吵架就分居，分居一两个月老公可以对我和孩子不问不闻，到时间了他就自己又搬回来，但是回来也不和我沟通，就算是上次是他动手，他也不道歉，他特别喜欢冷战。现在和老公又发展到了无性，而且双方家庭也合不来，我和他家人不敢来往，他家人对我要求高，希望天天在家做饭带娃伺候老公，他现在和我父母也不来往，从来不回家吃饭，要等我父母走了他才回来，每天很晚回家，去哪从来不告诉我，我知道他有婚外性，但他從來不承認，还说我是神经病，我還被他傳播上了高危型的HPV，以后是死是活都不确定，我特别想要离婚，我感觉和他生活快要窒息了，他整天拉着个脸，要么对我不闻不问，要么一开口说话就是责怪的语气，但他就是不离婚，说是为了孩子好，还让我不要发神经，我真的不知道该怎么和他相处了。他性格很古怪'
    ),
    (
        '妻子在钱的问题上不坦诚，如何处理好家庭经济问题？',
        '和老婆因为意外怀孕结婚，两家谈婚论嫁时闹得不愉快，我家出了一套大户型的房子，一台豪华车（当然这些都是婚前财产），然后给了40万装修婚房，同时给女方10万彩礼。而女方一开始的态度是一分钱没有，后来迫于压力给了50万现金，我老婆名下既没有房也没有车。婚后的钱一直都是给我老婆管理，包括我的工资奖金还有结婚的份子钱等等。我老婆并没有把这些钱看做是家庭财产，就算其中已经掺入了很多我的钱，还是认为全部都是她自己的钱，我只是知道她的卡密码而已，但是卡在哪里，她花了多少钱，花在哪里了，有没有借钱给别人，我通通不知道，这让我很不安。况且她没有理财观念，我曾经三番五次跟她说了理财的重要性，她就只是沉默不语，也不知道是听不懂，还是装傻实际上自己去做了理财但是没把收入情况告诉我，我们多次吵架都是为了钱，我觉得夫妻两个在这个问题上不能坦诚是个大问题，我现在很想经济分开，但是不知道怎么做才比较妥当'
    ),
    (
        '30岁男生，相亲屡次不顺，我该怎么办呢？',
        '30岁男生还单身，相亲对象不少，可是次次都以失败收场，自己长相还行，就是不太会说话，和女生聊天刚开始聊的挺不错，没聊几天女的就不回复了，爱答不理，心里郁闷的晚上都想哭，我该怎么办'
    ),
    (
        '30岁女生，离婚念头挥之不去，他为何计划婚内出轨？',
        '此刻有点儿失眠心情烦躁，我想了很久没有想明白，为什么他会计划着婚内出轨，……心里，我基本上认定原因应该是我身材不怎么好，胸部不够吸引人，因为婚后发现了在婚前，他约P了一个又肥又长相不佳的女人。我以为就是因为对方胸部傲人。在壹心理一些文章的引导下，我突然明白，源头是性生活不和谐。婚前同居一年多，到婚后2年，性生活基本就是，直奔主题，他一手拿着手机欣赏着他的成人影片一边爱爱，很少会主动关心我的感受，也很少理会我让他收起手机的要求。而且另一方面，他基本每晚都会要求我用嘴亲亲，他自己则拿着手机看片或者无关紧要的东西，让我经常觉得自己就是一个娃娃一个工具。以前我主动跟他说起过几次，然而也只是不了了之。后来我基本对性生活没有兴趣，也很反感为他口J。而且，日常生活变得频繁小吵冷战，很少有之前无话不谈的亲密，我不知道我们还有没有重归于好的机会……'
    ),
    (
        '29岁哺乳期，和公婆因带孩子问题每天崩溃，怎么办？',
        '怀孕她就没怎么管，备产各种东西都是自己准备。产后直接是妈妈照顾的。产后半年回来要上班，我爸妈又没退休，无奈下还是他们来照顾。他们方法不对，连个衣服都洗不干净，更不用提孩子的饭了。每天我都自己趁下班时间给孩子准备饭和各种东西。最怕的是习惯，比如她喜欢喂孩子各种炒菜，我说好几次一岁之内吃盐不好就是不听，还故意喂。或者孩子的奶瓶，经常不盖盖子让他把玩，说不卫生，也不搭理。孩子吃饭的碗和勺子永远是扔在各种桌子椅子角落，用的时候直接拿起来用也不洗，洗脸洗脚水每次告诉温度差不多就好了，就是不听，偏要一会冷一会烫。来我家被子也不叠，锅永远是不会洗的，下一顿加水加米接着煮。我老公又每天忙的不在家，我已经要崩溃了'
    ),
    (
        '婚后分居三四年，有过家庭矛盾，我的婚姻是否该继续？',
        '婚后分居三四年，和老公之间发生过一些家庭矛盾，可以看我另外一篇疑问。我们上半年闹离婚，后来他又主动和好，但是我们之前除了新婚，几乎很少有夫妻生活，他说一靠近我就觉得心里像一堆蚂蚁抓挠，特别难受。他说这是他得心理问题，小时候受到唾骂和凌辱，造成了心理阴影。我们俩都觉得生活的很痛苦，但为了孩子没有离婚。我跟他在一起就觉得拘束，不自在。向往的温馨家庭生活也得不到，我很累，我应该坚持离婚吗？他这种心理疾病是真的还是借口呢？'
    ),
]


In [15]:
n_gen = 3

reset_generating_args(args)
try:
#     args.recompute = True
    for title, text in input_texts:
        context_string = f'{title}<sep>{text}<sep><sep>'
#         context_string += '<|endoftext|>'
        print(title)
        print()
        print(text)
        print('-' * 100)
        print()
        for i in range(n_gen):
            args.top_p=random.gauss(0.5, 0.5)
            args.temperature=random.gauss(1, 0.05)
            print(f'{i+1}:\t', end='')
            context_tokens = tokenizer.EncodeAsIds(context_string).tokenization
            for id_ in infer_tokens_generative(context_tokens, model, tokenizer):
                s = tokenizer.DecodeIds([id_])
                print(s, end='')
            print(os.linesep)
        print()
        print('=' * 100)
        print()
finally:
    reset_generating_args(args)

男人出轨

我的意思是，出轨者会问自己，我为什么要这么做，被背叛的伴侣会问，你为什么要这样对我?
----------------------------------------------------------------------------------------------------

1:	<|endoftext|>离婚也许是一种一个取舍感,挫败感,对亲密感,以及你的吸引力。第三,自己的生活要自己承担责任就有。女人都是重感情的,建议你给这个男人客观的建议,就是你自己的感受,而不是去应对外人给予的。我想你可以再深入的沟通一下,说不定什么时候成为你和他之间的性格特点,以及你如何改变都会是你接受的底线。 <eos>

2:	<|endoftext|>关系封闭了我们很容易存续的阴影,感觉危险并没有怎么样。很容易互相我看到他的隐私,因为什么不这个事件还是想看看。 <eos>

3:	<|endoftext|>你得先杂志和夫妻之间学会如何化解紧张,把自己思维处理好了,越来越晴上天建议你去做一些独立的呼吸对,然后再争吵,引起意外的爱。每个人都是需要爱的,不需要经历肌肤之亲。也可以去回忆你们如何下去的?他出轨如何,而且婚姻从来都不是一个人的事情,需要双方都认可的伴侣。 <eos>



我的奇葩婚姻，老公说为了孩子好就是不离婚？

我的婚姻最近好像一塌糊涂，可是老公不离婚，我甚至有想过死？我是不是得了抑郁症？和老公从2017年开始就吵吵闹闹，一吵架就分居，分居一两个月老公可以对我和孩子不问不闻，到时间了他就自己又搬回来，但是回来也不和我沟通，就算是上次是他动手，他也不道歉，他特别喜欢冷战。现在和老公又发展到了无性，而且双方家庭也合不来，我和他家人不敢来往，他家人对我要求高，希望天天在家做饭带娃伺候老公，他现在和我父母也不来往，从来不回家吃饭，要等我父母走了他才回来，每天很晚回家，去哪从来不告诉我，我知道他有婚外性，但他從來不承認，还说我是神经病，我還被他傳播上了高危型的HPV，以后是死是活都不确定，我特别想要离婚，我感觉和他生活快要窒息了，他整天拉着个脸，要么对我不闻不问，要么一开口说话就是责怪的语气，但他就是不离婚，说是为了孩子好，还让我不要发神经，我真的不知道该怎么和他相处了。他性格很古怪
------------------------------------

## Test

使用 test 语料，从中随机打断，并预测下文，比较原文与预测结果！

随机选 N 个

In [26]:
from datetime import datetime
ts = datetime.now().strftime('%y%m%d%H%M%S')


N = 200
SHUFFLING = False
INFER_COUNT = 5

input_file = './data/xinliqa-hunyin/test.json'
output_file = f'./data/xinliqa-hunyin/test.infer-{HPARAMS_NAME}.{iteration_dir}-{N}x{args.out_seq_length}x{INFER_COUNT}-shuffle_{SHUFFLING}-{ts}.tsv'

print(f'output_file={output_file}')

output_file=./data/xinliqa-hunyin/test.infer-345m.iter_0050000-200x256x5-shuffle_False-191125120008.tsv


In [28]:
total = sum(1 for _ in tqdm(open(input_file)))
print(f'Test 数据总数: {total:,d}')

assert total >= N

print(f'Test 采样数: {N:,d}')

mask = np.zeros(total, dtype=int)
mask[:N] = 1
if SHUFFLING:
    np.random.shuffle(mask)

samples = []
with open(input_file) as fp:
    reader = compress(fp, mask)
    for line in tqdm(reader, 'sample', total=N):
        line = line.strip()
        if not line:
            continue
        text = json.loads(line)['text']
        question, answer = text.strip().split('<|endoftext|>')
        question_title, question_text = question.split('<sep>')[:2]
        samples.append([question_title, question_text, answer])
if SHUFFLING:
    random.shuffle(samples)

with open(output_file, 'w') as fp:
    writer = csv.writer(fp, delimiter='\t')
    for question_title, question_text, answer in tqdm(samples, 'generate'):
        row = [question_title, question_text, answer]
        txt = f'{question_title}<sep>{question_text}<sep><sep><|endoftext|>'
        for _ in range(INFER_COUNT):
            reset_generating_args(args)
            try:
                args.recompute=True
                args.top_p=random.gauss(0.5, 0.5)
                args.temperature=random.gauss(1, 0.05)
                infer_txt = ''.join(infer_text_generative(txt, model, tokenizer))
                row.append(infer_txt)                
            finally:
                reset_generating_args(args)
        writer.writerow(row)


Test 数据总数: 1,885
Test 采样数: 200
